<a href="https://colab.research.google.com/github/specM7/DSGP_Group_33_Brain_Tumor_Predictor/blob/Meningioma_Adrian_2425482/Meningioma_Adrian_2425482.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 AI-Powered Brain Tumor Predictor for Meningioma(Adrian)


## Project Goal
Build a Convolutional Neural Network (CNN) to classify brain MRI images into 4 tumor categories.

## Technologies Used
- **TensorFlow/Keras** - Deep Learning Framework
- **OpenCV** - Image Processing
- **NumPy** - Numerical Computing


## Model Specifications
- **Input**: 224x224 grayscale MRI images
- **Architecture**: Custom CNN with 3 convolutional blocks
- **Output**: 4-class classification (tumor types)
- **Training**: 25 epochs with validation split

###  Objectives
1. Build a robust CNN model for brain tumor classification
2. Achieve >90% accuracy on test data
3. Provide interpretable results with visualization

In [ ]:
import os, glob, cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

print("TensorFlow:", tf.__version__)


##  Extract Dataset
Unzips your file and shows folder structure.

---

## 2️ Dataset Preparation

###  Upload Dataset
Upload your `DataSet.zip` file containing:
```
DataSet/
├── Training/
│   ├── glioma/
│   ├── meningioma/
│   ├── notumor/
│   └── pituitary/
└── Testing/
    ├── glioma/
    ├── meningioma/
    ├── notumor/
    └── pituitary/
```

**Click the upload button below to select your dataset ZIP file.**

In [ ]:
from google.colab import files

uploaded = files.upload()        # PICK the real DataSet.zip from your PC

zip_name = list(uploaded.keys())[0]
print("Uploaded:", zip_name)

# Unzip into /content/DataSet_raw
!rm -rf /content/DataSet_raw
!mkdir -p /content/DataSet_raw
!unzip -q "$zip_name" -d /content/DataSet_raw

print("\nTop level under /content/DataSet_raw:")
!ls /content/DataSet_raw

print("\nAny 'Training' folders:")
!find /content/DataSet_raw -maxdepth 5 -type d -iname "Training"


##  Set Paths
Look at previous output, find your Training folder path, and paste it in next cell.

Example: `/content/DataSet_raw/DataSet/Training`

In [ ]:

training_path = "/content/DataSet_raw/DataSet/Training"   # example, edit to your real one

raw_root = os.path.dirname(training_path)
print("training_path:", training_path)
print("raw_root:", raw_root)
print("RAW training folders:", os.listdir(training_path))

img_size = (224, 224)
batch_size = 32

train_dir = training_path
test_dir  = os.path.join(raw_root, "Testing")
print("train_dir:", train_dir)
print("test_dir:", test_dir)





In [ ]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ────────────────────────────────────────────────
#   CLEAN DATASET CREATION – NO DUPLICATES
# ────────────────────────────────────────────────

train_ds = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse',
    color_mode='grayscale',
    subset='training',
    seed=42
)

val_ds = val_test_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse',
    color_mode='grayscale',
    subset='validation',
    seed=42
)

test_ds = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='sparse',
    color_mode='grayscale',
    shuffle=False,
    seed=42
)

print("Classes / indices:", train_ds.class_indices)
print("Number of training batches:", len(train_ds))
print("Number of validation batches:", len(val_ds))
print("Number of test batches:", len(test_ds))



## 6️ Model Evaluation

###  Testing on Unseen Data

Now we'll evaluate the model on completely unseen test data to measure real-world performance.

**Test Set:** Independent images never seen during training

In [ ]:
print("Meningioma class ID:", train_ds.class_indices['meningioma'])

In [ ]:
model = models.Sequential([
    layers.Input(shape=(*img_size, 1)),
    layers.Rescaling(1./255),

    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),

    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),

    layers.Conv2D(128, 3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),

    layers.Conv2D(256, 3, padding='same', activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=[
        "accuracy",
        tf.keras.metrics.Recall(class_id=train_ds.class_indices['meningioma'], name='meningioma_recall'),
        tf.keras.metrics.Precision(class_id=train_ds.class_indices['meningioma'], name='meningioma_precision')
    ]
)

model.summary()

# Optional but strongly recommended: class weights
from sklearn.utils import class_weight
import numpy as np

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_ds.classes),
    y=train_ds.classes
)
class_weight_dict = dict(enumerate(class_weights))

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=6,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6
    )
]

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,               # more epochs is ok now because of early stopping
    class_weight=class_weight_dict,
    callbacks=callbacks
)

# Unpack all four metrics returned by model.evaluate
test_loss, test_acc, test_meningioma_recall, test_meningioma_precision = model.evaluate(test_ds)
print("Test accuracy:", test_acc)
print("Test Meningioma Recall:", test_meningioma_recall)
print("Test Meningioma Precision:", test_meningioma_precision)

##  Save & Export Model

###  Saving Trained Model

The model will be saved in HDF5 format (.h5) which includes:
- ✅ Model architecture
- ✅ Trained weights
- ✅ Optimizer state
- ✅ Training configuration

**File:** `brain_tumor_mri_model.h5`

In [ ]:
model.save("brain_tumor_mri_model.h5")


In [ ]:
from google.colab import files
files.download("brain_tumor_mri_model.h5")
